In [ ]:
from keras.layers import Input, Lambda, Dense,Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [ ]:
IMAGE_SIZE = [224,224]

train_path = 'chest_xray/train'
valid_path  = 'chest_xray/test'

In [ ]:
vgg= VGG16(input_shape = IMAGE_SIZE +[3], weights = 'imagenet',include_top = False)

In [ ]:
for layer in vgg.layers:
    layer.trainable = False


In [ ]:
folders = glob("chest_xray/train/*")

In [ ]:
x = Flatten()(vgg.output)

In [ ]:
prediction = Dense(len(folders),activation='softmax')(x)
model = Model(inputs=vgg.input,outputs=prediction)

In [ ]:
model.summary()